In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

#Ovde se uvoze biblioteke koje će se koristiti u kodu. 
#pandas za rad sa podacima, numpy za numeričke operacije, 
#torch za rad s PyTorch tenzorima, 
#BertForQuestionAnswering za BERT model koji odgovara na pitanja, i BertTokenizer za tokenizaciju.

In [17]:
coqa = pd.read_json('http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json')
pd.set_option('display.max_colwidth', None)
coqa.head()

#Učitavaju se podaci iz CoQA skupa podataka koji su u JSON formatu. 
#coqa.head() se koristi da prikaže prvih nekoliko redova kako biste videli kako izgledaju podaci

version  \
0        1   
1        1   
2        1   
3        1   
4        1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [3]:
del coqa["version"]

#Izbacuje se kolona "version" iz DataFrame-a, 



In [4]:

cols = ["text","question","answer"]
#definišu se potrebne kolone za novi DataFrame (new_df): "text", "question", "answer".


comp_list = []
for index, row in coqa.iterrows():
    for i in range(len(row["data"]["questions"])):
        temp_list = []
        temp_list.append(row["data"]["story"])
        temp_list.append(row["data"]["questions"][i]["input_text"])
        temp_list.append(row["data"]["answers"][i]["input_text"])
        comp_list.append(temp_list)
        
        
new_df = pd.DataFrame(comp_list, columns=cols) 
#Pravi se novi DataFrame (new_df) koji sadrži tekst priče, pitanje i odgovor. 
#Podaci se čuvaju u CSV fajlu pod nazivom "CoQA_data.csv".

new_df.to_csv("CoQA_data.csv", index=False)
#cuvanje dataframe-a u csv fajl za buduce ucitavanje

In [5]:
data = pd.read_csv("CoQA_data.csv")
#Ucitavanje podataka iz lokalnog CSV fajla
data.head()


,text,question,answer
0,"The Vatican Apostolic Library (), more commonl...",When was the Vat formally opened?,It was formally established in 1475
1,"The Vatican Apostolic Library (), more commonl...",what is the library for?,research
2,"The Vatican Apostolic Library (), more commonl...",for what subjects?,"history, and law"
3,"The Vatican Apostolic Library (), more commonl...",and?,"philosophy, science and theology"
4,"The Vatican Apostolic Library (), more commonl...",what was started in 2014?,a project


In [15]:

#print(data["text"].head())


In [16]:
print("Number of question and answers: ", len(data))

Number of question and answers:  108647


In [18]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

#Učitavaju se prethodno obučeni BERT model i odgovarajući tokenizator.

C:\Users\Boris\miniconda3\envs\myenv\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Boris\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [19]:
"""random_num = np.random.randint(0,len(data))
question = data["question"][random_num]
text = data["text"][random_num]"""

In [20]:
"""input_ids = tokenizer.encode(question, text)
print("The input has a total of {} tokens.".format(len(input_ids)))"""

The input has a total of 354 tokens.


In [21]:
"""tokens = tokenizer.convert_ids_to_tokens(input_ids)
for token, id in zip(tokens, input_ids):
    print('{:8}{:8,}'.format(token,id))"""

[CLS]        101
how        2,129
many       2,116
of         1,997
their      2,037
own        2,219
large      2,312
areas      2,752
did        2,106
they       2,027
reside    13,960
in         1,999
?          1,029
[SEP]        102
long       2,146
beach      3,509
is         2,003
a          1,037
city       2,103
on         2,006
the        1,996
pacific    3,534
coast      3,023
of         1,997
the        1,996
united     2,142
states     2,163
,          1,010
within     2,306
the        1,996
greater    3,618
los        3,050
angeles    3,349
area       2,181
of         1,997
southern   2,670
california   2,662
.          1,012
as         2,004
of         1,997
2010       2,230
,          1,010
its        2,049
population   2,313
was        2,001
46         4,805
##2        2,475
,          1,010
257       24,368
.          1,012
it         2,009
is         2,003
the        1,996
36th      21,460
most       2,087
populous  20,151
city       2,103
in         1,999
the       

In [22]:
"""
#first occurence of [SEP] token
sep_idx = input_ids.index(tokenizer.sep_token_id)
print("SEP token index: ", sep_idx)
#number of tokens in segment A (question) - this will be one more than the sep_idx as the index in Python starts from 0
num_seg_a = sep_idx+1
print("Number of tokens in segment A: ", num_seg_a)
#number of tokens in segment B (text)
num_seg_b = len(input_ids) - num_seg_a
print("Number of tokens in segment B: ", num_seg_b)
#creating the segment ids
segment_ids = [0]*num_seg_a + [1]*num_seg_b
#making sure that every input token has a segment id
assert len(segment_ids) == len(input_ids)"""

SEP token index:  13
Number of tokens in segment A:  14
Number of tokens in segment B:  340


In [23]:
#token input_ids to represent the input and token segment_ids to differentiate our segments - question and text
"""output = model(torch.tensor([input_ids]),  token_type_ids=torch.tensor([segment_ids]))"""

In [24]:
#tokens with highest start and end scores
"""answer_start = torch.argmax(output.start_logits)
answer_end = torch.argmax(output.end_logits)
if answer_end >= answer_start:
    answer = " ".join(tokens[answer_start:answer_end+1])
else:
    print("I am unable to find the answer to this question. Can you please ask another question?")
    
print("\nQuestion:\n{}".format(question.capitalize()))
print("\nAnswer:\n{}.".format(answer.capitalize()))"""


Question:
How many of their own large areas did they reside in?

Answer:
Three.


In [25]:
def question_answer(question, text):
    # Tokenizacija pitanja i teksta kao para
    input_ids = tokenizer.encode(question, text)

    # Pretvaranje tokena u stringove
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Pronalaženje prvog pojavljivanja tokena [SEP]
    sep_idx = input_ids.index(tokenizer.sep_token_id)

    # Broj tokena u segmentu A (pitanje)
    num_seg_a = sep_idx + 1

    # Broj tokena u segmentu B (tekst)
    num_seg_b = len(input_ids) - num_seg_a

    # Lista nula i jedinica za segmentne ugnježdenosti
    segment_ids = [0] * num_seg_a + [1] * num_seg_b
    assert len(segment_ids) == len(input_ids)

    # Prolazak kroz model koristeći input_ids i segment_ids
    output = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))

    # Rekonstrukcija odgovora
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)

    if answer_end >= answer_start:
        answer = tokens[answer_start]

        # Spajanje tokena koji počinju sa "##"
        for i in range(answer_start + 1, answer_end + 1):
            if tokens[i][0:2] == "##":
                answer += tokens[i][2:]
            else:
                answer += " " + tokens[i]

    # Ako odgovor počinje sa "[CLS]", znači da odgovor nije pronađen
    if answer.startswith("[CLS]"):
        answer = "Unable to find the answer to your question."

    # Prikaz predviđenog odgovora
    print("\nPredicted answer:\n{}".format(answer.capitalize()))


In [33]:
text = """New York (CNN) -- More than 80 Michael Jackson collectibles -- including the late pop star's 
famous rhinestone-studded glove from a 1983 performance -- were auctioned off Saturday, reaping a total $2 million. 
Profits from the auction at the Hard Rock Cafe in New York's Times Square crushed pre-sale expectations of only 
$120,000 in sales. The highly prized memorabilia, which included items spanning the many stages of Jackson's career, 
came from more than 30 fans, associates and family members, who contacted Julien's Auctions to sell their gifts and 
mementos of the singer. Jackson's flashy glove was the big-ticket item of the night, fetching $420,000 from a buyer 
in Hong Kong, China. Jackson wore the glove at a 1983 performance during \"Motown 25,\" an NBC special where he 
debuted his revolutionary moonwalk. Fellow Motown star Walter \"Clyde\" Orange of the Commodores, who also performed 
in the special 26 years ago, said he asked for Jackson's autograph at the time, but Jackson gave him the glove instead.
"The legacy that [Jackson] left behind is bigger than life for me,\" Orange said. \"I hope that through that glove 
people can see what he was trying to say in his music and what he said in his music.\" Orange said he plans to give a 
portion of the proceeds to charity. Hoffman Ma, who bought the glove on behalf of Ponte 16 Resort in Macau, paid a 25 
percent buyer's premium, which was tacked onto all final sales over $50,000. Winners of items less than $50,000 paid a 
20 percent premium."""

question = "Where was the Auction held?"
question_answer(question, text)
#original answer from the dataset
print("Original answer:\n", data.loc[data["question"] == question]["answer"].values[0])


Predicted answer:
Hard rock cafe in new york ' s times square
Original answer:
 Hard Rock Cafe


In [34]:
question = "What is the capital of France?"
question_answer(question, text)


Predicted answer:
Unable to find the answer to your question.


In [36]:
#Prikaz rada funkcije question_answer kroz interakciju sa korisnikom

text = input("Please enter your text: \n")
question = input("\nPlease enter your question: \n")
while True:
    question_answer(question, text)
    
    flag = True
    flag_N = False
    
    while flag:
        response = input("\nDo you want to ask another question based on this text (Y/N)? ")
        if response[0] == "Y":
            question = input("\nPlease enter your question: \n")
            flag = False
        elif response[0] == "N":
            print("\nBye!")
            flag = False
            flag_N = True
            
    if flag_N == True:
        break

Please enter your text: 
Dr. Dimitrije D. Chvokić is an associate professor at the Faculty of Natural Sciences and Mathematics (FNSM) at the University of Banja Luka. Previously, he worked as a teaching associate (assistant and senior assistant) at the same university from 2010 to 2021. In his role as an associate, he conducted exercises primarily in computer science subjects. At his initiative, syllabi of subjects such as Chemical Informatics (SP Chemistry), Fundamentals of Informatics (SP Chemistry), Computer Applications in Biology (SP Biology), and Operating Systems (SP TViI) underwent changes during his collaboration.  According to available data, he holds a record for the number of subjects (assignments) and weekly workload at the University of Banja Luka. His areas of interest include optimization at two levels, mathematical modeling of strategic decision-making, prediction of dynamic changes, and ledger technologies (blockchain). Additional information is available on the Biogr